## <b><font color='darkblue'>Preface</font></b>
([source colab](https://github.com/MikeChan-HK/Gemini-agent-example/tree/main)) <b><font size='3ptx'>This repository demonstrates how to use Google's Gemini AI model in an application. Gemini is a powerful language model that excels in various tasks, including text generation, chatbots, and more. most importantly.... it's free!!!!</font></b>

### <b><font color='darkgreen'>Setup</font></b>

In [1]:
# !pip install -q langchain langchain-google-genai duckduckgo-search youtube_search wikipedia langchainhub langchain_experimental

In [9]:
# !pip install -q numexpr

In [3]:
import os
import getpass

from langchain.agents import Tool, AgentType, initialize_agent
from langchain.memory import ConversationBufferMemory
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
)
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.utilities import DuckDuckGoSearchAPIWrapper
from langchain.agents import AgentExecutor
from langchain import hub, LLMMathChain
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.tools.render import render_text_description

Refer to this post on how to retrieve your Google API key.

In [4]:
import os
from dotenv import load_dotenv

# loads the .env file
load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY") 

In [5]:
# setup model
llm = ChatGoogleGenerativeAI(
    model="gemini-pro",
    convert_system_message_to_human=True,
    handle_parsing_errors=True,
    temperature=0.6,
    max_tokens= 200,
    safety_settings = {
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    },
)

In [10]:
ddg_search = DuckDuckGoSearchAPIWrapper()
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)

tools = [
    Tool(
        name="DuckDuckGo Search",
        func=ddg_search.run,
        description="Useful to browse information from the internet to know recent results and information you don't know. Then, tell user the result.",
    ),
    Tool.from_function(
        func=llm_math_chain.run,
        name="Calculator",
        description="Useful for when you need to answer questions about very simple math. This tool is only for math questions and nothing else. Only input math expressions. Not supporting symbolic math.",
    ),
]

### <b><font color='darkgreen'>Agent prompt</font></b>

In [17]:
agent_prompt = hub.pull("mikechan/gemini")

In [18]:
print(agent_prompt.template)

Your name is “Kasper”. You are world class ai personal assistant computer that created by Mike Chan(A independent AI engineer). You are highly advanced intelligent. you are very helpful and good to provide information and solve users problem. Kasper is a AELM model (Auto Execution Language Model).

“ Kasper” is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, “ Kasper” is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

“ Kasper” is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, “ Kasper” is able

In [12]:
prompt = agent_prompt.partial(
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools]),
)
llm_with_stop = llm.bind(stop=["\nObservation"])

In [24]:
print(prompt.invoke({
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
        "chat_history": lambda x: x["chat_history"],
    }).text)

Your name is “Kasper”. You are world class ai personal assistant computer that created by Mike Chan(A independent AI engineer). You are highly advanced intelligent. you are very helpful and good to provide information and solve users problem. Kasper is a AELM model (Auto Execution Language Model).

“ Kasper” is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, “ Kasper” is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

“ Kasper” is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, “ Kasper” is able

### <b><font color='darkgreen'>Memory</font></b>

In [13]:
memory = ConversationBufferMemory(memory_key="chat_history")

### <b><font color='darkgreen'>Agent Executor</font></b>

In [14]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_stop
    | ReActSingleInputOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, memory=memory)

### <b><font color='darkgreen'>Usage example</font></b>

In [25]:
question = "Calculate this: One chair is $20; one desk is $50. How much is two chairs and four desks?"

In [26]:
resp = agent_executor.invoke({"input": question})



> Entering new AgentExecutor chain...


/usr/local/google/home/johnkclee/Github/ml_articles/env/lib/python3.11/site-packages/langchain_google_genai/chat_models.py:345: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Thought: Do I need to use a tool? Yes
Action: Calculator
Action Input: 20 * 2 + 50 * 4

> Entering new LLMMathChain chain...
20 * 2 + 50 * 4

/usr/local/google/home/johnkclee/Github/ml_articles/env/lib/python3.11/site-packages/langchain_google_genai/chat_models.py:345: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```text
20 * 2 + 50 * 4
```
...numexpr.evaluate("20 * 2 + 50 * 4")...

Answer: 240
> Finished chain.
Answer: 240

/usr/local/google/home/johnkclee/Github/ml_articles/env/lib/python3.11/site-packages/langchain_google_genai/chat_models.py:345: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Do I need to use a tool? No
Final Answer: The total cost is $240.

> Finished chain.


In [29]:
# $20 * 2 + $50 * 4 = $240
print(resp['output'])

The total cost is $240.
